# Detect Model
## Step 3 - Deploy Endpoint as TensorFlow Model

ref:  https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/


In [ ]:
import boto3, re, os, sys
from sagemaker import get_execution_role
from sagemaker.tensorflow.model import TensorFlowModel

from PIL import Image
import numpy as np

import tensorflow as tf

In [ ]:
# This is needed since we cloned tensorflow/models under code.
cwd = os.getcwd()
models = os.path.join(cwd, 'code/models/research/')
slim = os.path.join(cwd, 'code/models/research/slim')
sys.path.append(models)
sys.path.append(slim)

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
role = get_execution_role()

In [ ]:
S3_SAGEMAKER = "s3://sagemaker-us-east-1-586454201570/"
SAGEMAKER_JOB = "cfa-products-mobilenet-v1-SSD-2019-08-03-20-36-17-231/"
MODEL_OUTPUT = "output/model.tar.gz"
IMAGE_SIZE = (300,300)

FULL_MODEL_PATH = S3_SAGEMAKER + SAGEMAKER_JOB + MODEL_OUTPUT
! aws s3 ls {FULL_MODEL_PATH}

SAMPLE_IMAGE = "/home/ec2-user/SageMaker/ssd-dag/data/new_jpeg_images/20190710_variety_1562781002.jpg"

# NAME - get this from the console
ENDPOINT_NAME = "ep-mobilenet-ssd"

## Endpoint

The SageMaker console appears to be the best place to create your SageMaker endpoint.    If you did NOT train your model on SageMaker (shame on you), you'll have to jump through some hoops.    I'm not covering that here because training on SageMaker seems to have many advantages but -- it seems you have to bundle up your model artifacts in a tarball that mimics the output of a SageMaker HOSTED training session.

### Endpoint  Configuration
Use the console to create an endpoint configuration.   This includes specifying the EC2 instance type.  Name your instance with a ep_* prefix to make it recognizable.   Include model and instance type.   For example:  

ep-config-mobilenet-ssd-p2xlarge-gpu

It appears you can install multiple models (?) - you can put more details in the model config line.

### Create Endpoint from a Model

Not sure about specifying a model in the configuration and the endpoint - need to read more documentation.   Name your endpoint:  

ep-mobilenet-ssd


In [ ]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel

sagemaker_session = sagemaker.Session()
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(ENDPOINT_NAME, sagemaker_session)

### Create an Endpoint with code

Below is an example of creating an endpoint with code - referencing an S3 bucket where the model artifacts are stored.   This is way harder - use SageMaker to training    your model and avoid this hassle.    PLUS - it failed after 20 minutes when I tried it!

In [ ]:
# 20190804
# DON'T EVEN THINK ABOUT THE OBVIOUS!
#  py_version='py3'
#  framework_version='1.13'

sagemaker_model = TensorFlowModel(model_data = FULL_MODEL_PATH,
                    role = role,
                    framework_version='1.12',
                    entry_point = 'code/train.py')

In [ ]:
# predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')

## Grab an Image

In [ ]:
# Helper Function
# load an image and resturn a numpy array

def load_image_into_numpy_array(image):
  (im_width, im_height) = IMAGE_SIZE
  image = image.resize((im_width, im_height), Image.ANTIALIAS)
  image = np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)
  return image

In [ ]:
image = Image.open(SAMPLE_IMAGE)
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)

print (image_np_expanded.shape)

In [ ]:
tensor_image = tf.constant(image_np_expanded, shape=(1,300,300,3), dtype=tf.uint8, name="image_tensor")
print (tensor_image)

In [ ]:
tensor_num_detections = tf.compat.v1.placeholder(tf.float32, shape=None, name="num_detections")
tensor_detection_boxes = tf.compat.v1.placeholder(tf.float32, shape=None, name="detection_boxes")
tensor_detection_scores = tf.compat.v1.placeholder(tf.float32, shape=None, name="detection_scores")
tensor_detection_classes = tf.compat.v1.placeholder(tf.float32, shape=None, name="detection_classes")

tensor_dict = {'num_detections': tensor_num_detections, 
              'detection_boxes': tensor_detection_boxes, 
              'detection_scores': tensor_detection_scores, 
              'detection_classes': tensor_detection_classes}
print (tensor_dict)

In [ ]:
tensor_dict: {'num_detections': <tf.Tensor 'num_detections:0' shape=<unknown> dtype=float32>, 
              'detection_boxes': <tf.Tensor 'detection_boxes:0' shape=<unknown> dtype=float32>, 
              'detection_scores': <tf.Tensor 'detection_scores:0' shape=<unknown> dtype=float32>, 
              'detection_classes': <tf.Tensor 'detection_classes:0' shape=<unknown> dtype=float32>}


In [ ]:
image_tensor = tf.constant(value=image_np_expanded, shape=(1, 300, 300, 3), dtype=tf.uint8)
feed_dict={image_tensor: image_np_expanded}

In [ ]:
%%time
# output_dict = predictor.predict({"tf_example": image_np_expanded})
output_dict = predictor.predict(image_tensor)